In [12]:
import random
import math

def vecteur_site(offre, capacites, matrice_distance):
    vecteur_resultant = []
    for i, cap_site in enumerate(offre):
        vecteur_site = []
        for j, cap_poste in enumerate(capacites):
            if cap_poste >= cap_site and matrice_distance[i][j] <= 30:
                vecteur_site.append(j + 1)  # Ajouter l'indice du poste en commençant par 1
            else:
                vecteur_site.append(-1)  # Ajouter -1 si non raccordable
        vecteur_resultant.append(vecteur_site)
    return vecteur_resultant

def shuffle(vecteurs_sites):
    vecteurs_sites_ameliore = []
    for vecteur_site in vecteurs_sites:
        random.shuffle(vecteur_site)  # Mélanger l'ordre des valeurs dans chaque vecteur
        vecteurs_sites_ameliore.append(vecteur_site)
    return vecteurs_sites_ameliore

def creation_solution(vecteur_resultant, offre, capacites, demande):
    solution = []
    capacites_restantes = capacites[:]  # Copie des capacités
    demande_restante = demande
    for i, vecteur in enumerate(vecteur_resultant):
        site_assigne = False
        for site in vecteur:
            if site != -1 and capacites_restantes[site - 1] >= offre[i]:
                solution.append(site)
                capacites_restantes[site - 1] -= offre[i]
                demande_restante -= offre[i]
                site_assigne = True
                break
        if not site_assigne:
            solution.append(-1)  # Aucun site valide trouvé ou capacité insuffisante
    return solution, capacites_restantes, demande_restante

def afficher_vecteurs(vecteurs, titre):
    print(titre)
    for i, vecteur in enumerate(vecteurs):
        print(f"Vecteur pour le site {i+1}: {vecteur}")

def teste_realisabilites(solution, capacites, demande):
    if any(cap < 0 for cap in capacites):
        return False, "Solution non réalisable car la capacité de ce poste est insuffisante."
    if demande > 0:
        return False, "Solution non réalisable car la demande n'est pas satisfaite."
    return True, "Solution réalisable."

def rectification(offre, capacites_restantes, solution, demande_restante):
    while demande_restante < 0:
        min_offre_index = -1
        min_offre_value = float('inf')
        for i, o in enumerate(offre):
            if o > 0 and o < min_offre_value and solution[i] != -1 and solution[i] != 0:
                min_offre_index = i
                min_offre_value = o

        if min_offre_index == -1:  # Si aucun site valide n'est trouvé
            break

        poste_index = solution[min_offre_index] - 1  # Indice du poste actuellement assigné

        if abs(demande_restante) <= min_offre_value:
            # Si la demande restante peut être couverte par l'offre minimale
            offre[min_offre_index] -= abs(demande_restante)  # Ajuster l'offre du site
            capacites_restantes[poste_index] += abs(demande_restante)  # Restaurer la capacité utilisée au poste
            demande_restante += abs(demande_restante)  # Mettre à jour la demande restante à 0
            if offre[min_offre_index] == 0:
                solution[min_offre_index] = 0  # Désactiver ce site dans la solution
        else:
            # Si la demande restante est supérieure à l'offre minimale
            offre[min_offre_index] = 0  # Effacer l'offre du site désactivé pour éviter sa réutilisation
            capacites_restantes[poste_index] += min_offre_value  # Restaurer toute la capacité utilisée
            demande_restante += min_offre_value  # Réduire la demande restante
            solution[min_offre_index] = 0  # Désactiver ce site

    return offre, capacites_restantes, solution, demande_restante

def fitness(solution, matrice_distance):
    fitness_value = 0
    for i, site in enumerate(solution):
        if site != -1 and site != 0 :
            fitness_value += matrice_distance[i][site - 1]
    return fitness_value

def mise_a_jour(solution1, solution2, matrice_distance):
    fitness1 = fitness(solution1, matrice_distance)
    fitness2 = fitness(solution2, matrice_distance)
    
    if fitness2 < fitness1:
        return solution2, fitness2
    else:
        return solution1, fitness1

sites = ["Abadla", "Kenadsa", "Aflou", "Nakhla", "Taleb Larbi", "Touggourt", "Leghrous", "Tolga", "Khenguet Sidi Nadji", "Batmete", "M Ghaier", "Guerrara"]
offre = [2666650, 3999900, 6666600, 6666600, 2666650, 4333200, 6666600, 2666650, 4999800, 7333200, 7333200, 266650]

# Définir les dimensions de la matrice

postes = ["GME", "sebdou", "Z'bair", "Tlemcen", "Remchi", "Beni Saf", "Ain Timouchent 1", "Ain Timouchent 2", "S.Yacoub", "Boutlelis", "Messerghine", "Oran Sud", "Bousfer", "Sidi el kbir", "Sidi bel Abbes", "Telagh", "sidi bel abbes ville", "mecheria", "Mekera", "hadjout", "Bouhanifia", "draa ben khedda", "ain sekhouna", "El bayadh", "tizi medane", "mostaganem port", "mostaganem port sablette", "ain tadeles", "relizane", "tissemsilte", "Bordj bou naama", "mahdia", "khemis miliana", "mouzaia", "Bechar", "HSO \"il estfermé\"", "pvn", "PSD", "el biodh sidi cheikh", "sidi ali", "tadjena", "kherba", "O,fodda"]

capacites = [6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 6000000, 600000]
donnees = list(zip(postes, capacites))
# Définir les dimensions de la matrice
nb_lignes = len(offre)
nb_colonnes = len(capacites)
matrice_distance = [[random.randint(10, 50) for _ in range(nb_colonnes)] for _ in range(nb_lignes)]

demande = 80000000
matrice_distance = [[random.randint(10, 50) for _ in range(nb_colonnes)] for _ in range(nb_lignes)]


# Initialisation des variables pour la boucle
taux_initial = 5
taux_final = 4
alpha = random.uniform(0.8, 0.99)
nbr_iterations = 5
taux = taux_initial

# Liste pour stocker toutes les solutions parcourues
solutions_parcourues = []

# Calcul des vecteurs de sites et création de la solution initiale
vecteurs_sites = vecteur_site(offre, capacites, matrice_distance)
solution_initiale, capacites_restantes_init, demande_restante_init = creation_solution(vecteurs_sites, offre, capacites, demande)

# Calcul de la fitness de la solution initiale
fitness_initiale = fitness(solution_initiale, matrice_distance)

# Initialisation de la meilleure solution et de la meilleure fitness
best_solution = solution_initiale
best_fitness = fitness_initiale

# Affichage de la solution initiale et de sa fitness
print("Solution initiale:", solution_initiale)
print("Fitness initiale:", fitness_initiale)

# Boucle de mise à jour des solutions
while taux > taux_final:
    for i in range(nbr_iterations):
        # Amélioration des vecteurs de sites
        vecteurs_sites_ameliore = shuffle(vecteurs_sites)
        
        # Création d'une nouvelle solution à partir des vecteurs de sites améliorés
        nouvelle_solution, capacites_restantes_amel, demande_restante_amel = creation_solution(vecteurs_sites_ameliore, offre, capacites, demande)
        
        # Rectification si la demande est négative
        if demande_restante_amel < 0:
            offre_modifiee, capacites_restantes_amel, nouvelle_solution, demande_restante_amel = rectification(
                offre.copy(), capacites_restantes_amel.copy(), nouvelle_solution.copy(), demande_restante_amel
            )
        
        # Calcul de la fitness de la nouvelle solution
        nouvelle_fitness = fitness(nouvelle_solution, matrice_distance)
        
        # Ajout de la solution à la liste des solutions parcourues
        solutions_parcourues.append((nouvelle_solution, nouvelle_fitness))
        
        # Calcul du delta de fitness
        delta_fitness = nouvelle_fitness - fitness_initiale
        
        # Mise à jour de la solution selon le delta de fitness
        if delta_fitness <= 0:
            best_solution, best_fitness = mise_a_jour(best_solution, nouvelle_solution, matrice_distance)
            vecteurs_sites = vecteurs_sites_ameliore
        else:
            p = random.uniform(0, 1)
            if 0 <= p <= math.exp(-delta_fitness / taux):
                best_solution, best_fitness = mise_a_jour(best_solution, nouvelle_solution, matrice_distance)
                vecteurs_sites = vecteurs_sites_ameliore
    
    # Mise à jour du taux
    taux *= alpha
# Affichage de la meilleure solution trouvée et de sa fitness
print("Meilleure solution trouvée:", best_solution)
print("Meilleure fitness:", best_fitness)

# Affichage de toutes les solutions parcourues et de leurs fitness
#print("Toutes les solutions parcourues et leurs fitness :")
#for solution, fitness_value in solutions_parcourues:
    #print("Solution:", solution, "Fitness:", fitness_value)


Solution initiale: [4, 3, -1, -1, 1, 8, -1, 4, 2, -1, -1, 2]
Fitness initiale: 134
Meilleure solution trouvée: [28, 34, -1, -1, 29, 36, -1, 15, 37, -1, -1, 28]
Meilleure fitness: 108
